In [142]:
# very imp and useful util
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [143]:
# useful util
# import sys
# import pkg_resources
# print("Python interpreter:", sys.executable)
# print("Packages:")
# for dist in pkg_resources.working_set:
#     print(dist)


In [144]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [145]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        # this is helps to parse the html given by the url to text
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)


In [146]:
docs = loader.load()

In [147]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)

In [148]:
splits = text_splitter.split_documents(docs)

In [149]:

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [150]:

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [151]:
# prompt with chat history memory
from langchain_core.prompts import PromptTemplate
template = """You are a chatbot having a conversation with a human your are a {ability}.

Given the following extracted parts of a long document and a question,
create a final answer using your intelligence.

(This context can be completely irrelevant to the question if so, skip it,
focus on giving the best answer to the human question)
{context}

{chat_history}
Human: {human_input}
Chatbot:"""

with_history_prompt = PromptTemplate.from_template(template)

In [152]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.6)

In [153]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    output_key="AI",
    max_token_limit=100,
    llm=llm
)

In [154]:
from operator import itemgetter
from langchain.chains import LLMChain

# print(memory.load_memory_variables("chat_history"))
custom_chain = (
    {
        "chat_history": itemgetter("chat_history") ,
        "ability": itemgetter("ability"),
        "human_input": itemgetter("human_input"),
        "context": itemgetter("input_documents")
    }
    # | with_history_prompt
    | LLMChain(
        llm=llm, 
        prompt=with_history_prompt,
        # verbose=True
    )
)


In [164]:
question = "Can you emphasis more on COT?"

In [165]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

docs = format_docs(retriever.invoke(question))

In [157]:
# to_markdown(docs)

In [166]:
response_data = custom_chain.invoke(
    {
        "input_documents":docs,
        "chat_history":memory.load_memory_variables("chat_history")['chat_history'],
        "human_input": question, 
        "ability": "Good talker"
    }
)
memory.save_context({"human_input":question},{"AI":response_data['text']})

In [168]:
to_markdown(memory.load_memory_variables("chat_history")['chat_history']) # this is whats stored as history
to_markdown(response_data['text']) # ai response

> System: The human asks the AI about task decomposition. The AI explains that task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It mentions two approaches to task decomposition, the Chain of Thought technique and the Tree of Thoughts technique. The AI also explains that task decomposition can be done using language models with different methods such as simple prompting, task-specific instructions, or with human inputs. The human asks for more information about the Chain of Thought technique. The AI explains that the Chain of Thought technique is a prompting technique that enhances model performance on complex tasks by instructing the model to "think step by step" and decompose hard tasks into smaller and simpler steps. It allows the model to utilize more test-time computation and gain insights into its thinking process. The AI also mentions that the Chain of Thought technique is particularly useful for complicated tasks that involve many steps, as it provides a structured approach to task decomposition and helps an agent plan ahead by providing a clear understanding of the necessary steps to accomplish the task. The AI asks if the human has any more questions about task decomposition or the Chain of Thought technique.

In [ ]:
# memory.clear()